<a href="https://colab.research.google.com/github/Nanfemicit/EOS-Course/blob/main/Catchment_analysis_exersise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pysheds

In [ ]:
import rasterio
from rasterio.transform import from_bounds

from pysheds.grid import Grid

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
import numpy as np
import rasterio
import heapq
import matplotlib.pyplot as plt
import gdown

# Download data

In [ ]:
#'GOOGLE_DRIVE_LINK' to TIFF file
url = 'https://drive.google.com/file/d/1ZS_dK3FfGm8syGeDppD1eu9m9aiCKPlM/view?usp=drive_link'
# Download the TIFF file
file_path = 'dem_Tenerife.tif'
gdown.download(url, file_path, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1ZS_dK3FfGm8syGeDppD1eu9m9aiCKPlM
To: /content/dem_Tenerife.tif
100%|██████████| 57.9M/57.9M [00:00<00:00, 109MB/s]


'dem_Tenerife.tif'

# Load Digital Elevation Model (DEM)

In [ ]:
#Load DEM file
dem_file = 'dem_Tenerife.tif'
# Load the DEM data
grid = Grid.from_raster(dem_file, data_name='dem')
dem = grid.read_raster(dem_file)
grid

# Prepare DEM for flow analysis

In [ ]:
# Condition DEM
# ----------------------
# Fill pits in DEM

# Fill depressions in DEM

# Resolve flats in DEM

# Analyse flow directions and accumulation

In [ ]:
# Compute flow directions - D8 flow directions
# -------------------------------------


In [ ]:
# Calculate flow accumulation
# --------------------------


We can use the flow accumulation field and load it in GIS to delinate the pure points in our study area

In [ ]:
# We define a flow accumulation threshold to only extract the major stream network
threshold = 1000  # Number of cells contributing to flow
streams = (acc > threshold)*1.0  # Binary mask for stream network

#Save flow accumulation
grid.to_raster(streams,'flow_accumulation.tif')

# Delineate your catchment

In [ ]:
#-----------------------------------------------------------
# Delineate a catchment
# ----------------------------------------------------------
#set pour point
lon = 1
lat = 1
#-----------------------------------------------------------

In [ ]:
# Snap pour point to high accumulation cell
x_snap, y_snap = grid.snap_to_mask(acc > 1000, (lon, lat))

# Delineate the catchment
catch = grid.catchment(x=x_snap, y=y_snap, fdir=fdir, xytype='coordinate')

In [ ]:
# Crop and plot the catchment
# ---------------------------
# Clip the bounding box to the catchment
grid.clip_to(catch)
clipped_catch = grid.view(catch)

In [ ]:
#Save catchment mask
grid.to_raster(clipped_catch.astype(np.int32),'catchment.tif')

# Determine hydrological charateritics for the catchment

In [ ]:
# Calculate distance to outlet from each cell
# -------------------------------------------
dist = grid.distance_to_outlet(x=x_snap, y=y_snap, fdir=fdir, xytype='coordinate')

In [ ]:
# Compute flow directions - D8 flow directions only for catchment
# -------------------------------------
fdir = grid.flowdir(inflated_dem)

# A simple flow time model

In [ ]:
#Specify grid cell size (here 20m)
L = 20 #length (size) of a grid cell

In [ ]:
#drop in height to downstream cell

In [ ]:
#plot the estimated flow time map

# Generate a flow rate ("discharge") diagram

In [ ]:
# Flatten the 2D array to a 1D list
flow_times = acc_t.flatten()

# Remove NaN or infinite values
flow_times = flow_times[np.isfinite(flow_times)]

# Choose time step and unit conversion (options: "seconds", "minutes", "hours")
time_unit = "hours"  # Change to "seconds" or "hours" as needed

if time_unit == "seconds":
    time_step = 10  # Bin width in seconds
    time_label = "Time to Outlet (seconds)"
elif time_unit == "minutes":
    flow_times = flow_times / 60  # Convert to minutes
    time_step = 1  # Bin width in minutes
    time_label = "Time to Outlet (minutes)"
elif time_unit == "hours":
    flow_times = flow_times / 3600  # Convert to hours
    time_step = 0.1  # Bin width in hours
    time_label = "Time to Outlet (hours)"

# Create bins for histogram
max_time = np.max(flow_times)
bins = np.arange(0, max_time + 2 * time_step, time_step)

# Compute histogram of travel times
discharge, bin_edges = np.histogram(flow_times, bins=bins)

# Get time values (bin centers)
times = bin_edges[:-1]

In [ ]:
# Plot the discharge curve
plt.figure(figsize=(10, 5))
plt.plot(times, discharge, marker='o', linestyle='-', color='b', label="Discharge")
plt.xlabel(time_label)
plt.ylabel("Flow rate [cells]")
plt.title("Flow Rate Curve Over Time")
plt.grid(True)
plt.legend()
plt.show()